# Template 

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
%load_ext watermark

In [2]:
import pandas as pd

In [3]:
# Run this cell before close.
%watermark
%watermark --iversion
%watermark -b -r -g

2020-05-19T02:18:10+00:00

CPython 3.7.7
IPython 7.14.0

compiler   : GCC 8.3.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : 
CPU cores  : 16
interpreter: 64bit
pandas 1.0.3

Git hash: 7c7c2ede423997d8b3fa48946dbe1518ca64d4e8
Git repo: https://github.com/ysraell/examples.git
Git branch: master


In [4]:
df_train = pd.read_csv('train.csv')
df_test_ori = pd.read_csv('test.csv')
df_test = pd.read_csv('test.csv')

df = df_train[list(df_test.columns)]
df['NU_NOTA_MT'] = df_train['NU_NOTA_MT']
corr = df.corr()
feat_order = sorted(corr['NU_NOTA_MT'].fillna(0).to_dict().items(), key=lambda x: x[1], reverse=True)
cols_feat = [x[0] for x in feat_order if x[1] >= 0.2]
cols_feat

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


['NU_NOTA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_REDACAO',
 'TP_DEPENDENCIA_ADM_ESC',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP5',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP1']

In [5]:
df_train['NU_NOTA_MT'].unique()

array([399.4, 459.8,   nan, ..., 787.1, 824.6, 634.1])

In [6]:
df = df_train[list(df_test.columns)]
df['NU_NOTA_MT'] = df_train['NU_NOTA_MT']
df = df[cols_feat]
df_test = df_test[cols_feat[1:]]
target = 'NU_NOTA_MT'
df = df.fillna(0).reset_index(drop=True)
features = pd.get_dummies(df)
import numpy as np

# Labels are the values we want to predict
train_labels = np.array(features[target])
# Remove the labels from the features
# axis 1 refers to the columns
train_features = features.drop(target, axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features_nparray = np.array(features)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)


Training Features Shape: (13730, 10)
Training Labels Shape: (13730,)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 2000, random_state = 30, n_jobs = -1, verbose = 1)
# Train the model on training data
rf.fit(train_features, train_labels);

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:   17.1s finished


In [25]:
test_features = pd.get_dummies(df_test.fillna(0))
predictions = rf.predict(test_features)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 1218 tasks      | elapsed:    0.7s
[Parallel(n_jobs=16)]: Done 1768 tasks      | elapsed:    0.9s
[Parallel(n_jobs=16)]: Done 2000 out of 2000 | elapsed:    1.0s finished


In [26]:
predictions

array([431.0388 , 440.5988 , 584.0324 , ..., 682.7069 , 478.19515,
         0.     ])

In [27]:
df_test_ori['NU_NOTA_MT'] = list(predictions)
col_send = ['NU_INSCRICAO','NU_NOTA_MT']

In [28]:
df_test_ori[col_send].head()

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,431.03880
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,440.59880
2,b38a03232f43b11c9d0788abaf060f7366053b6d,584.03240
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,0.00000
4,715494628a50142ce8cb17191cfe6d0f3cae0934,495.82305


In [29]:
df_test_ori[col_send].to_csv('answer.csv',index=False)